# Modelo NLP: Endpoint en Tiempo Real.

In [ ]:
import numpy as np
import pandas as pd

import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'module_5/part_3'

- Usaremos el estimador que entrenamos anterioremente en el notebook [module_4/4_05.ipynb](../module_4/4_05.ipynb).
- Podemos usar el método de clase attach de *Estimator* con el *training_job_name* usado anteriormete para no tener que realizar el entrenamiento otra vez.

In [ ]:
estimator = sagemaker.estimator.Estimator.attach(training_job_name='dbpedia-blazingtext', )

- Desplegamos nuestro modelo

In [ ]:
text_classifier = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge", 
    serializer=sagemaker.serializers.JSONSerializer()
)

- Tenemos que realizar el mismo procesamiento que en proceso de entrenamiento.

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
sentences = [
    "Convair was an american aircraft manufacturing company which later expanded into rockets and spacecraft.",
    "Berwick secondary college is situated in the outer melbourne metropolitan suburb of berwick .",
]

# using the same nltk tokenizer that we used during data preparation for training
tokenized_sentences = [" ".join(nltk.word_tokenize(sent)) for sent in sentences]
tokenized_sentences

- BlazingText admite application/json como content-type para la inferencia. 
- El contenido tiene que incluir la lista de frases con la clave *instances*.

In [ ]:
payload = {"instances": tokenized_sentences}
response = text_classifier.predict(payload)
response

In [ ]:
import json
predictions = json.loads(response)
print(json.dumps(predictions, indent=2))

- Por defecto solo nos da  la clase con mayor probabilidad, pero podemos modificarlo.

In [ ]:
payload = {"instances": tokenized_sentences, "configuration": {"k": 2}}

response = text_classifier.predict(payload)

predictions = json.loads(response)
print(json.dumps(predictions, indent=2))

In [ ]:
text_classifier.delete_endpoint()